<a href="https://colab.research.google.com/github/maciejgorczak/phishing/blob/master/phishing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Phishing
####my approach to classification (supervised learning) of phishing sites using deep neural networks and pytorch library.
https://archive.ics.uci.edu/ml/datasets/phishing+websites



In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [7]:
#loading the data from .xlsx(previously read in Excel as it was more approachable than reading straight from .arff file)  
import pandas as pd
data =pd.read_excel('/content/gdrive/My Drive/phising.xlsx')
print(data.head())

   @attribute having_IP_Address    ...  @attribute Result  
0                              -1  ...                   -1
1                               1  ...                   -1
2                               1  ...                   -1
3                               1  ...                   -1
4                               1  ...                    1

[5 rows x 31 columns]


In [0]:
import numpy as np
import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.utils.data
from scipy.stats import pearsonr

In [10]:
#looking at value distribution
print(data['@attribute Result  '].value_counts())
data.head()

 1    6157
-1    4898
Name: @attribute Result  , dtype: int64


,@attribute having_IP_Address,@attribute URL_Length,@attribute Shortining_Service,@attribute having_At_Symbol,@attribute double_slash_redirecting,@attribute Prefix_Suffix,@attribute having_Sub_Domain,@attribute SSLfinal_State,@attribute Domain_registeration_length,@attribute Favicon,@attribute port,@attribute HTTPS_token,@attribute Request_URL,@attribute URL_of_Anchor,@attribute Links_in_tags,@attribute SFH,@attribute Submitting_to_email,@attribute Abnormal_URL,@attribute Redirect,@attribute on_mouseover,@attribute RightClick,@attribute popUpWidnow,@attribute Iframe,@attribute age_of_domain,@attribute DNSRecord,@attribute web_traffic,@attribute Page_Rank,@attribute Google_Index,@attribute Links_pointing_to_page,@attribute Statistical_report,@attribute Result
0,-1,1,1,1,-1,-1,-1,-1,-1,1,1,-1,1,-1,1,-1,-1,-1,0,1,1,1,1,-1,-1,-1,-1,1,1,-1,-1
1,1,1,1,1,1,-1,0,1,-1,1,1,-1,1,0,-1,-1,1,1,0,1,1,1,1,-1,-1,0,-1,1,1,1,-1
2,1,0,1,1,1,-1,-1,-1,-1,1,1,-1,1,0,-1,-1,-1,-1,0,1,1,1,1,1,-1,1,-1,1,0,-1,-1
3,1,0,1,1,1,-1,-1,-1,1,1,1,-1,-1,0,0,-1,1,1,0,1,1,1,1,-1,-1,1,-1,1,-1,1,-1
4,1,0,-1,1,1,-1,1,1,-1,1,1,1,1,0,0,-1,1,1,0,-1,1,-1,1,-1,-1,0,-1,1,1,1,1


In [11]:
#is data complete?
print(data.isnull().any())

@attribute having_IP_Address               False
@attribute URL_Length                      False
@attribute Shortining_Service              False
@attribute having_At_Symbol                False
@attribute double_slash_redirecting        False
@attribute Prefix_Suffix                   False
@attribute having_Sub_Domain               False
@attribute SSLfinal_State                  False
@attribute Domain_registeration_length     False
@attribute Favicon                         False
@attribute port                            False
@attribute HTTPS_token                     False
@attribute Request_URL                     False
@attribute URL_of_Anchor                   False
@attribute Links_in_tags                   False
@attribute SFH                             False
@attribute Submitting_to_email             False
@attribute Abnormal_URL                    False
@attribute Redirect                        False
@attribute on_mouseover                    False
@attribute RightClic

In [12]:
#changinmg the names of columns
cols = []
for col in data.columns:
  col = col.split(" ")[1]
  col = col.split("_")
  if type(col)==list:
    colo = []
    for elem in col:
      if elem.isupper() == False:
        elem = elem.capitalize()
        colo.append(elem)
      else:
        colo.append(elem)
    colo = " ".join(colo) 
    cols.append(colo)

data.columns = cols
data.head()

,Having IP Address,URL Length,Shortining Service,Having At Symbol,Double Slash Redirecting,Prefix Suffix,Having Sub Domain,Sslfinal State,Domain Registeration Length,Favicon,Port,HTTPS Token,Request URL,URL Of Anchor,Links In Tags,SFH,Submitting To Email,Abnormal URL,Redirect,On Mouseover,Rightclick,Popupwidnow,Iframe,Age Of Domain,Dnsrecord,Web Traffic,Page Rank,Google Index,Links Pointing To Page,Statistical Report,Result
0,-1,1,1,1,-1,-1,-1,-1,-1,1,1,-1,1,-1,1,-1,-1,-1,0,1,1,1,1,-1,-1,-1,-1,1,1,-1,-1
1,1,1,1,1,1,-1,0,1,-1,1,1,-1,1,0,-1,-1,1,1,0,1,1,1,1,-1,-1,0,-1,1,1,1,-1
2,1,0,1,1,1,-1,-1,-1,-1,1,1,-1,1,0,-1,-1,-1,-1,0,1,1,1,1,1,-1,1,-1,1,0,-1,-1
3,1,0,1,1,1,-1,-1,-1,1,1,1,-1,-1,0,0,-1,1,1,0,1,1,1,1,-1,-1,1,-1,1,-1,1,-1
4,1,0,-1,1,1,-1,1,1,-1,1,1,1,1,0,0,-1,1,1,0,-1,1,-1,1,-1,-1,0,-1,1,1,1,1


In [0]:
#distribution of unique values in all atributes in the dataset including the decisive one
'''
for col in data.columns:
  print(data[col].value_counts())
'''
#data is well distributed through the dataset

In [0]:
#preparing data for correlation calculations
attributes = data.drop("Result", axis=1)
target = data["Result"]
attributes_cor = attributes.columns.tolist()
target_cor = target.name

In [37]:
#Pearson correlation coefficient - measure of linear correlation between given attribute and result
correlations = {}
for d in attributes_cor:
    data_temp = data[[d,target_cor]]
    x1 = data_temp[d].values
    x2 = data_temp[target_cor].values
    key = d + ' vs ' + target_cor
    correlations[key] = float(pearsonr(x1,x2)[0])
data_correlations = pd.DataFrame(correlations, index=['Result']).T
data_correlations.loc[data_correlations['Result'].abs().sort_values(ascending=False).index]
print(data_correlations[data_correlations['Result'] < 0.00])

                                         Result
Shortining Service vs Result          -0.067966
Double Slash Redirecting vs Result    -0.038608
Domain Registeration Length vs Result -0.225789
Favicon vs Result                     -0.000280
HTTPS Token vs Result                 -0.039854
Abnormal URL vs Result                -0.060488
Redirect vs Result                    -0.020113
Iframe vs Result                      -0.003394


In [0]:
#deleting the attributes not beeing correlated at all with the result
columns_to_drop = ['Domain Registeration Length', 'Shortining Service', 'Abnormal URL', 'HTTPS Token', 'Double Slash Redirecting',
                   'Redirect', 'Iframe', 'Favicon',]
data = data.drop(columns_to_drop, axis=1)

In [39]:
print(data.head())

   Having IP Address  URL Length  ...  Statistical Report  Result
0                 -1           1  ...                  -1      -1
1                  1           1  ...                   1      -1
2                  1           0  ...                  -1      -1
3                  1           0  ...                   1      -1
4                  1           0  ...                   1       1

[5 rows x 23 columns]


In [40]:
#changing the values from -1&1 to 0&1 to prepare data to upload them to NN
data['Decisive'] = 1
data.Decisive[data.Result < 0] = 0
data.head()
data.drop(columns='Result', inplace=True)
data.head

<bound method NDFrame.head of        Having IP Address  URL Length  ...  Statistical Report  Decisive
0                     -1           1  ...                  -1         0
1                      1           1  ...                   1         0
2                      1           0  ...                  -1         0
3                      1           0  ...                   1         0
4                      1           0  ...                   1         1
...                  ...         ...  ...                 ...       ...
11050                  1          -1  ...                   1         1
11051                 -1           1  ...                   1         0
11052                  1          -1  ...                   1         0
11053                 -1          -1  ...                   1         0
11054                 -1          -1  ...                  -1         0

[11055 rows x 23 columns]>

In [41]:
#splitting data into training and test datasets
mask = np.random.rand(len(data)) < 0.75
train = data[mask]
test = data[~mask]
print(train.shape, test.shape)

(8313, 23) (2742, 23)


In [52]:
'''
almost manual tuning of neural network
'''
hiperparameters = [32, 64, 128]
batch = [30, 50, 70]
learning = [0.007, 0.009, 0.01]
di = {}
for i in hiperparameters:
    '''
    hiperparameters of the neural network
    '''
    inputSize = len(train.columns) -1  
    hidden1Size = i 
    hidden2Size = i 
    numClasses = 2 
    numEpoch = 100 
    print(i)
    bat = {}
    for b in batch:
        batchSize = b 
        print(b)
        ler = {}
        for l in learning:
            learningRate = l 
            print(l)
            
            trainLoader = torch.utils.data.DataLoader(dataset=torch.tensor(train.values), batch_size=batchSize, shuffle=True)
            testLoader = torch.utils.data.DataLoader(dataset=torch.tensor(test.values))

            '''
            below model of the neural network
            '''
            class DeepNeuralNetwork(nn.Module):
                def __init__(self, inputSize, hidden1Size, hidden2Size, numClasses):
                    super(DeepNeuralNetwork, self).__init__()
                    self.fc1 = nn.Linear(inputSize, hidden1Size)
                    self.relu1 = nn.ReLU()
                    self.fc2 = nn.Linear(hidden1Size, hidden2Size)
                    self.relu2 = nn.ReLU()
                    self.fc3 = nn.Linear(hidden2Size, numClasses)
                    self.logsm1 = nn.LogSoftmax(dim=1)
                    
                def forward(self, x):
                    out = self.fc1(x)
                    out = self.relu1(out)
                    out = self.fc2(out)
                    out = self.relu2(out)
                    out = self.fc3(out)
                    out = self.logsm1(out)
                    return out

            dnn = DeepNeuralNetwork(inputSize, hidden1Size, hidden2Size, numClasses)

            #loss function and optimiizer
            lossFN = nn.NLLLoss()
            optimizer = torch.optim.Adam(dnn.parameters(), lr=learningRate)

            #training
            for epoch in range(0, numEpoch):
                for i, data in enumerate(trainLoader,0):
                    labels = Variable(data[:,-1])
                    data = Variable(data[:,0:22].float())
                    optimizer.zero_grad()
                    outputs = dnn(data)
                    loss = lossFN(outputs, labels.long())
                    loss.backward()
                    optimizer.step() 
                                                    
                #print('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, numEpoch, loss.item()))

            #checking the model on test data
            correct = 0
            total = 0
            for data in testLoader:
                labels = Variable(data[:,-1])
                data = Variable(data[:,0:22].float()) 
                outputs = dnn(data)
                _, predicted = torch.max(outputs.data, 1)  
                total += labels.size(0)
                correct += int((predicted == labels.long()).sum())
            print('Accuracy of the network on the data: {:.4f}%'.format(100 * correct / total))
            bas = float("{:.4f}".format(100 * correct / total))
            ler[l] = bas
        bat[b] = ler
    di[i] = bat

32
30
0.007
Accuracy of the network on the data: 95.9519%
0.009
Accuracy of the network on the data: 95.9519%
0.01
Accuracy of the network on the data: 96.0977%
50
0.007
Accuracy of the network on the data: 96.3530%
0.009
Accuracy of the network on the data: 95.5507%
0.01
Accuracy of the network on the data: 95.1860%
70
0.007
Accuracy of the network on the data: 96.0613%
0.009
Accuracy of the network on the data: 96.2436%
0.01
Accuracy of the network on the data: 95.8060%
64
30
0.007
Accuracy of the network on the data: 95.9519%
0.009
Accuracy of the network on the data: 96.0977%
0.01
Accuracy of the network on the data: 96.2801%
50
0.007
Accuracy of the network on the data: 96.3895%
0.009
Accuracy of the network on the data: 94.5660%
0.01
Accuracy of the network on the data: 95.9519%
70
0.007
Accuracy of the network on the data: 96.3530%
0.009
Accuracy of the network on the data: 95.9154%
0.01
Accuracy of the network on the data: 94.8213%
128
30
0.007
Accuracy of the network on the da

In [50]:
print(di)

{118: {30: {0.007: 96.1342, 0.009: 96.0977, 0.01: 96.353}, 50: {0.007: 96.3895, 0.009: 96.2436, 0.01: 96.6083}, 70: {0.007: 96.2801, 0.009: 96.1342, 0.01: 96.2801}}}
